In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
import numpy as np
import pyaudio
import time
import librosa
import wave
import queue
import IPython.display as ipd
from IPython.display import Audio
import soundfile as sf
import os
import threading
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
# Định nghĩa các hằng số
RATE = 16000
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
n_times_1s = int(RATE / CHUNK * 1)  # Number of iterations to record 1 second
n_times_145s = int(RATE*1.5/ CHUNK)  # Number of iterations to record 1 second
predict_mode = 0
p = pyaudio.PyAudio()

In [3]:
def features_extractor_1s(file_name):
    audio, sample_rate = librosa.load(file_name,sr=16000, mono=True)
    frame_length = int(0.025 * sample_rate)
    hop_length = int(0.015 * sample_rate)
    n_fft = 2 ** int(np.ceil(np.log2(frame_length)))
    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)
    desired_size = 66
    mfccs = np.zeros((13,desired_size))
    mfccs[:, :min(desired_size, mfcc.shape[1])] = mfcc[:, :min(desired_size, mfcc.shape[1])]
    delta_mfccs = librosa.feature.delta(mfccs)
    delta2_mfccs = librosa.feature.delta(mfccs, order=2)
    mfccs_features = np.concatenate((mfccs, delta_mfccs, delta2_mfccs))
    mfccs_features = np.transpose(mfccs_features)

    return mfccs_features

In [4]:
# Hàm trích xuất đặc trưng MFCC
def features_extractor_145s(file_name):
    audio, sample_rate = librosa.load(file_name,sr=16000, mono=True)
    frame_length = int(0.025 * sample_rate)
    hop_length = int(0.015 * sample_rate)
    n_fft = 2 ** int(np.ceil(np.log2(frame_length)))
    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)
    desired_size = 101
    mfccs = np.zeros((13,desired_size))
    mfccs[:, :min(desired_size, mfcc.shape[1])] = mfcc[:, :min(desired_size, mfcc.shape[1])]
    delta_mfccs = librosa.feature.delta(mfccs)
    delta2_mfccs = librosa.feature.delta(mfccs, order=2)
    mfccs_features = np.concatenate((mfccs, delta_mfccs, delta2_mfccs))
    mfccs_features = np.transpose(mfccs_features)

    return mfccs_features

In [5]:
def mic_check_1s(file_name):
    mfcc_processed = features_extractor_1s(filename)
    prediction = model_1s.predict(np.expand_dims(mfcc_processed, axis=0))
    if prediction[:, 0] > 0.99:
        print(f"Wake Word Detected for ({i})")
        print("Confidence:", prediction[:, 1])
        display(Audio(filename))
    return 0

In [6]:
# Hàm dự đoán số 
def mic_check_145s(file_name):
    global predict_mode
    mfccs_features = features_extractor_145s(file_name)
    # print(mfccs_features.shape())
    # data = mfccs_features
    data = np.stack(mfccs_features)
    data = np.reshape(data,(1,101,39))
    pre_test = model_145s.predict(data, verbose=0)
    print(pre_test)
    s=0
    result = 0
    for i in pre_test[0]:
        if i > 5e-1:
            result = i
            break
        s=s+1
    
    if s == 0:
        print("Predict: Batdieuhoa")
        display(Audio(get_file_path('Response','Batdieuhoa.wav'), rate=16000, autoplay=True))
    elif s == 1:
        print("Predict: Tatdieuhoa")
        display(Audio(get_file_path('Response','Tatdieuhoa.wav'), rate=16000, autoplay=True))
    elif s == 2:
        print("Predict: Tang1do")
        display(Audio(get_file_path('Response','Tang1do.wav'),  rate=16000,autoplay=True))
    # elif s == 3:
    #     print("Predict: Tang2do")
    # elif s == 4:
    #     print("Predict: Tang3do")
    elif s == 3:
        print("Predict: Giam1do")
        display(Audio(get_file_path('Response','Giam1do.wav'), rate=16000, autoplay=True))
    # elif s == 5:
    #     print("Predict: Giam2do")
    # elif s == 7:
    #     print("Predict: Giam3do")
    elif s == 4:
        print("Predict: Bat26do")
        display(Audio(get_file_path('Response','Bat26do.wav'), rate=16000, autoplay=True))
    elif s == 5:
        print("Predict: Silent")
    else:
        print("Unrecognizable")
    if s < 5:
        predict_mode = 0
        time.sleep(2)
    print('Reliability: %.2f %%' %(result*100.00))
    display(Audio(file_name))
    
#         print('Reliability: %.2f %%' %(result*100.00))
#         display(Audio(file_name))
    

In [7]:
def get_file_path(folder,file_name):
    # Lấy đường dẫn đến thư mục làm việc hiện tại trong Jupyter
    current_directory = os.getcwd()

    # Đường dẫn đến thư mục chứa tệp mô hình "H5"
    h5_folder_path = os.path.join(current_directory, folder)

    # Đường dẫn đến tệp mô hình được chỉ định
    file_path = os.path.join(h5_folder_path, file_name)

    return file_path

In [8]:
# Hàm của luồng 1
def record_audio_1s(file_number, stream):
    print(f"Recording audio for file {file_number}...")
    global predict_mode
    frames = []

    for _ in range(n_times_1s):
        # Lấy một khung dữ liệu
        data = stream.read(CHUNK)
        frames.append(data)

    # Lưu file âm thanh vào file wav
    filename =  "wakeup_{}".format(file_number)+".wav"
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
    return filename
        

In [9]:
# Hàm của luồng 1
def record_audio_145s(file_number, stream):
    print(f"Recording audio for file {file_number}...")
    frames = []

    for _ in range(n_times_145s):
        # Lấy một khung dữ liệu
        data = stream.read(CHUNK)
        frames.append(data)

    # Lưu file âm thanh vào file wav
    filename =  "speech_{}".format(file_number)+".wav"
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
    return filename

In [10]:
# Hàm của luồng 1
def record_audio():
    print("Recording audio...")
    global predict_mode
    #Khởi tạo Audio
    stream = p.open(format=FORMAT,
                    channels=1,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    for i in range(500):
        filename = record_audio_1s(i, stream)
        mfcc_processed = features_extractor_1s(filename)
        data = np.stack(mfcc_processed)
        data = np.reshape(data,(66,39,1))
        prediction = model_1s.predict(np.expand_dims(data, axis=0))
        print(prediction)
        if prediction[:, 0] > prediction[:, 1]:
            print(f"Wake Word Detected for ({i})")
            print("Confidence:", prediction[:, 1])
            display(Audio(get_file_path('Response','Hello.wav'), rate=16000, autoplay=True))
            display(Audio(filename))
            predict_mode = 0
            while (predict_mode < 10):
                speech = record_audio_145s(predict_mode, stream)
                mic_check_145s(speech)
                predict_mode += 1
                !del {speech}
            display(Audio(get_file_path('Response','Bye.wav'), rate=16000, autoplay=True))
        !del {filename}
        
        
    
    # Đóng stream và thoát khỏi PyAudio
    stream.stop_stream()
    stream.close()
    p.terminate()

    
      
    

In [11]:
model_145s = load_model(get_file_path("H5","Train_101_True_AIS_150_29_LSTM.h5"), compile=False)
# Au_AN_Rain
model_1s = load_model(get_file_path("WuW_H5","Train_66_WuW_True_AIS_1s.h5"), compile=False)

In [12]:
record_audio()

Recording audio...
Recording audio for file 0...
1/1 [==============================] - 0s 330ms/step
[[4.3976826e-14 1.0000000e+00]]
Recording audio for file 1...
1/1 [==============================] - 0s 31ms/step
[[1.5639158e-12 1.0000000e+00]]
Recording audio for file 2...
1/1 [==============================] - 0s 25ms/step
[[4.077530e-05 9.999592e-01]]
Recording audio for file 3...
1/1 [==============================] - 0s 16ms/step
[[0.9989886  0.00101315]]
Wake Word Detected for (3)
Confidence: [0.00101315]


Recording audio for file 0...
[[9.6691153e-12 2.6938919e-15 3.8714280e-15 1.5681147e-11 4.0942188e-05
  9.9995911e-01]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 1...
[[9.9999976e-01 2.4940837e-07 4.6991092e-18 5.1044632e-15 1.7698884e-12
  1.4298827e-13]]
Predict: Batdieuhoa


Reliability: 100.00 %


Recording audio for file 1...
[[2.0320877e-24 1.7018807e-16 3.0434442e-18 1.0792334e-17 7.8050934e-20
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


...

Recording audio for file 6...
[[3.0141206e-18 6.4464409e-17 2.3331626e-16 5.1296061e-15 4.0676244e-11
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 7...
[[4.3657820e-29 1.4682961e-21 4.5535541e-21 1.9896951e-21 1.9184181e-22
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 8...
[[1.1061331e-26 1.3108049e-15 3.9404469e-11 1.2382834e-15 1.4128731e-18
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 9...
[[8.0818696e-21 1.8450289e-16 2.4874708e-11 1.3713079e-12 9.0039121e-10
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 4...
1/1 [==============================] - 0s 41ms/step
[[1.8640094e-16 1.0000000e+00]]
Recording audio for file 5...
1/1 [==============================] - 0s 30ms/step
[[1.3016897e-17 1.0000000e+00]]
Recording audio for file 6...
1/1 [==============================] - 0s 29ms/step
[[1.1272673e-12 1.0000000e+00]]
Recording audio for file 7...
1/1 [==============================] - 0s 27ms/step
[[2.0361077e-16 1.0000000e+00]]
Recording audio for file 8...
1/1 [==============================] - 0s 25ms/step
[[1.1331701e-15 1.0000000e+00]]
Recording audio for file 9...
1/1 [==============================] - 0s 22ms/step
[[2.276122e-27 1.000000e+00]]
Recording audio for file 10...
1/1 [==============================] - 0s 25ms/step
[[1.8292907e-21 1.0000000e+00]]
Recording audio for file 11...
1/1 [==============================] - 0s 23ms/step
[[3.733937e-22 1.000000e+00]]
Recording audio for file 12...
1/1 [==============================] - 0s 25ms/step
[[1.553